In [17]:
#from json import loads
from kafka import KafkaConsumer
import pandas as pd
from pyspark.sql import SparkSession
import ast
import json

In [2]:
spark = SparkSession.builder.appName("Test").getOrCreate()

In [6]:
consumer = KafkaConsumer('swat_elte',bootstrap_servers=['kafka-server:9092'],
                         group_id=None,
                        auto_offset_reset='earliest', api_version=(0,10,1))

In [15]:
def json_deserializer(data):
    return json.dumps(data).decode("utf-8")

In [42]:
df_row = pd.json_normalize(json.loads(message))

In [50]:
df = pd.DataFrame()
i = 0

for message in consumer:
    message = message.value
    if len(message) < 10:
        break
    df_row = pd.json_normalize(json.loads(message))
    # df = df.append(df_row)
    df = pd.concat([df, df_row])
    if len(df) % 1000 == 0:
        i += 1000
        print("Rows Consumed: ", i)


Rows Consumed:  1000
Rows Consumed:  2000
Rows Consumed:  3000
Rows Consumed:  4000
Rows Consumed:  5000
Rows Consumed:  6000
Rows Consumed:  7000
Rows Consumed:  8000
Rows Consumed:  9000
Rows Consumed:  10000
Rows Consumed:  11000
Rows Consumed:  12000
Rows Consumed:  13000
Rows Consumed:  14000


In [51]:
print(df.shape)
df.head()

(14996, 78)


,GMT +0,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
0,2019-07-20 08:39:59,4.323736,492.896881,2,2,2,1,131.408615,9.313829,257.933868,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
0,2019-07-20 08:39:58,4.323736,492.465100,2,2,2,1,131.408615,9.316713,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
0,2019-07-20 08:39:57,4.303558,492.308100,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
0,2019-07-20 08:39:56,4.253915,491.405273,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
0,2019-07-20 08:39:55,4.200429,491.169769,2,2,2,1,131.408615,9.319918,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1


In [52]:
sdf = spark.createDataFrame(df)
sdf.show()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-------------------+-----------+----------+------+--------+-----------+-----------+----------+--------+----------+----------+--------+--------+--------+--------+-----+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+----------+----------+----------+----------+----------+------+------+------+------+--------+-----------+-----------+-------+----------+-----------+----------+--------+--------+-----------+-----------+-----------+-----------+-----+----------+----------+----------+----------+-----------+-----------+-----------+----------+------+------+------+------+--------+-----------+-----------+----------+----------+----------+----------+-------+-------+--------+--------+--------+-------+--------+-----------+-----------+-----------+
|             GMT +0|    FIT 101|   LIT 101|MV 101|P1_STATE|P101 Status|P102 Status|   AIT 201| AIT 202|   AIT 203|   FIT 201|  LS 201|  LS 202| LSL 203|LSLL 203|MV201|P2_STATE|P201 Status|P202 S

In [161]:
spark